# Fine Tuning Large Language Model - Model

In this workshop, you will learn how to fine tune the prompts and the LLMs to enhance and improves its response.

In [1]:
# Import libraries
import torch, time
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, GenerationConfig, TrainingArguments

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

In [29]:
# TODO: Load and explore the following datasets

dataset_name = "knkarthick/dialogsum"
model_name = "google/flan-t5-small"
model_name = "google/flan-t5-base"

dataset = load_dataset(dataset_name)

print(dataset)
print(dataset.shape)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})
{'train': (12460, 4), 'validation': (500, 4), 'test': (1500, 4)}


In [30]:
# TODO: Print a record
idx = 100
for k, v in dataset['train'][idx].items():
   print(f'{k}\n\t{v}')

id
	train_100
dialogue
	#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.
summary
	#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.
topic
	cable


In [31]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


## Fine tuning the LLM model

In this workshop we will be turning the <code>google/flan-t5-base</code> model.

In [24]:
# Utility function to dump a model's tunable parameters

def print_trainable_model_params(model):
   trainable_model_params = 0
   all_model_params = 0
   for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
         trainable_model_params += param.numel()
   return f"Trainable parameters: {trainable_model_params}\nTotal parameters: {all_model_params}\nPercentable of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [25]:
# TODO: Load model
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [26]:
# TODO: Print number of trainable parameters
print(print_trainable_model_params(original_model))

Trainable parameters: 247577856
Total parameters: 247577856
Percentable of trainable parameters: 100.00%


### Preprocess the dialogue dataset

We will train the model to summarize dialogue by creating a dialogue-summary pair for the LLM to process. The dialogue is the training data and the summary is the label. This is supervized learning.

The prompt will be as follows

```
Summarize the following dialogue.\n
\n
Fred: ...\n
Barney: ...\n
\n
Summary:\n
Summary of the conversation between Fred and Barney
```

The prompt and the summary will be tokenized for the LLM

In [27]:
# Utitlity function to prepare the data for training 
# Tokenize function
def tokenize_fn(data):
   start_prompt = 'Summarize the following dialogue.\n\n'
   end_prompt = '\n\nSummary:'
   prompt = [ start_prompt + d + end_prompt for d in data['dialogue'] ]
   summary = data['summary']
   data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
   data['labels'] = tokenizer(summary, padding="max_length", truncation=True, return_tensors="pt").input_ids
   return data


In [28]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


In [19]:
# TODO: prepare the data for training with the tokenize_fn function
tokenized_dataset = dataset.map(tokenize_fn, batched=True)



Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [20]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 1500
    })
})


In [34]:
# TODO: Verify prepared data
idx = 100
for k, v in tokenized_dataset['test'][idx].items():
   print(f'key={k}, value={v}')


key=id, value=test_33_2
key=dialogue, value=#Person1#: OK, that's a cut! Let's start from the beginning, everyone.
#Person2#: What was the problem that time?
#Person1#: The feeling was all wrong, Mike. She is telling you that she doesn't want to see you any more, but I want to get more anger from you. You're acting hurt and sad, but that's not how your character would act in this situation.
#Person2#: But Jason and Laura have been together for three years. Don't you think his reaction would be one of both anger and sadness?
#Person1#: At this point, no. I think he would react the way most guys would, and then later on, we would see his real feelings.
#Person2#: I'm not so sure about that.
#Person1#: Let's try it my way, and you can see how you feel when you're saying your lines. After that, if it still doesn't feel right, we can try something else.
key=summary, value=#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person

In [39]:
text = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'])
print(f'input_ids: {text}\n')
text = tokenizer.decode(tokenized_dataset['test'][idx]['labels'])
print(f'labels: {text}')

input_ids: Summarize the following dialogue. #Person1#: OK, that's a cut! Let's start from the beginning, everyone. #Person2#: What was the problem that time? #Person1#: The feeling was all wrong, Mike. She is telling you that she doesn't want to see you any more, but I want to get more anger from you. You're acting hurt and sad, but that's not how your character would act in this situation. #Person2#: But Jason and Laura have been together for three years. Don't you think his reaction would be one of both anger and sadness? #Person1#: At this point, no. I think he would react the way most guys would, and then later on, we would see his real feelings. #Person2#: I'm not so sure about that. #Person1#: Let's try it my way, and you can see how you feel when you're saying your lines. After that, if it still doesn't feel right, we can try something else. Summary:</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [40]:
# TODO: Remove id, dialogue, summary and topic columns from dataset. We only want input_ids and labels
tokenized_dataset = tokenized_dataset.remove_columns([ 'id', 'dialogue', 'summary', 'topic' ])

In [41]:
# TODO: Verify dataset again
idx = 100
for k, v in tokenized_dataset['test'][idx].items():
   print(f'key={k}, value={v}')

key=input_ids, value=[12198, 1635, 1737, 8, 826, 7478, 5, 1713, 345, 13515, 536, 4663, 10, 6902, 6, 24, 31, 7, 3, 9, 1340, 55, 1563, 31, 7, 456, 45, 8, 1849, 6, 921, 5, 1713, 345, 13515, 357, 4663, 10, 363, 47, 8, 682, 24, 97, 58, 1713, 345, 13515, 536, 4663, 10, 37, 1829, 47, 66, 1786, 6, 4794, 5, 451, 19, 5188, 25, 24, 255, 744, 31, 17, 241, 12, 217, 25, 136, 72, 6, 68, 27, 241, 12, 129, 72, 11213, 45, 25, 5, 148, 31, 60, 6922, 4781, 11, 6819, 6, 68, 24, 31, 7, 59, 149, 39, 1848, 133, 1810, 16, 48, 1419, 5, 1713, 345, 13515, 357, 4663, 10, 299, 9637, 11, 10591, 43, 118, 544, 21, 386, 203, 5, 1008, 31, 17, 25, 317, 112, 6363, 133, 36, 80, 13, 321, 11213, 11, 24784, 58, 1713, 345, 13515, 536, 4663, 10, 486, 48, 500, 6, 150, 5, 27, 317, 3, 88, 133, 8922, 8, 194, 167, 3413, 133, 6, 11, 258, 865, 30, 6, 62, 133, 217, 112, 490, 6382, 5, 1713, 345, 13515, 357, 4663, 10, 27, 31, 51, 59, 78, 417, 81, 24, 5, 1713, 345, 13515, 536, 4663, 10, 1563, 31, 7, 653, 34, 82, 194, 6, 11, 25, 54, 217, 14

### Tune model with pre-processed dataset

We will use [<code>Trainer</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#api-reference%20][%20transformers.Trainer) to train the original model. The training result will be written out. The trainer will be configure with [<code>TrainingArgument</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [ ]:
# CUDA information

print('CUDA available: ', torch.cuda.is_available())
if torch.cuda.is_available():
   print('B16 supported: ', torch.cuda.is_bf16_supported())
   torch.cuda.set_device(0)
   print('Current device: ', torch.cuda.current_device())
   print('CUDA device name: ', torch.cuda.get_device_name(0))

## Fine tuning the LLM Model with Low-Rank Adaptation (LoRA) / Parameter Efficient Fine Tuning (PEFT)

We will add a LoRA adapter to the LLM (flan-t5-base) and train the adapter. The original LLM will be frozen. The adapter can be combined with the original LLM during inferencing. 

In [ ]:
# TODO: Configure LoRA


In [ ]:
# TODO: Add LoRA to the LLM model to be trained


In [ ]:
# TODO: Print number of parameters, compare LoRA to the original model


In [ ]:
# TODO: Train model with LoRA


In [ ]:
# TODO: Create trainer and train model


### Use a trained LoRA model

The training will take a few hours and over many iterations.

For the purpose of this workshop we use a save model [intotheverse/peft-dialogue-summary-checkpoint](https://huggingface.co/intotheverse/peft-dialogue-summary-checkpoint).

In [ ]:
#TODO: Load the original model and add the pre-trained LoRA adaptation to the model
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'



## Evaluate LoRA model

In [ ]:
# TODO: Evaluate LoRA model against the original 



In [ ]:
# Prepare data for evaluation
dialogues = []
summaries = []
orig_model_summaries = []
lora_model_summaries = []
config = GenerationConfig(max_new_tokens=200)

for i in range(5):
   print(f'i = {i}')
   d = dataset['test'][i]['dialogue']
   s = dataset['test'][i]['summary']
   prompt = f"Summarize the following conversation.\n\n{d}\n\nSummary:"
   tokenized_prompt = tokenizer(prompt, return_tensors='pt').input_ids
   orig_resp = original_model.generate(input_ids=tokenized_prompt, generation_config=config)
   orig_resp_text = tokenizer.decode(orig_resp[0], skip_special_tokens=True)
   lora_resp = lora_model.generate(input_ids=tokenized_prompt, generation_config=config)
   lora_resp_text = tokenizer.decode(lora_resp[0], skip_special_tokens=True)

   summaries.append(s)
   orig_model_summaries.append(orig_resp_text)
   lora_model_summaries.append(lora_resp_text)

zipped_summaries = list(zip(summaries, orig_model_summaries, lora_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['label', 'original_model_summary', 'lora_model_summary'])
df

### Evaluate models with ROUGE/Bleu metrics

Recall-Oriented Understudy for Gisting Evaluate ([ROUGE](https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363)) is a set of metrics used to evaluate the quality of machine-generated text, such as summaries and translations. ROUGE metrics compare the generated text to a human-written reference and measure the overlap between the two. 

The metrics range between 0 and 1, with higher scores indicating higher similarity between the baseline and generated text.

In [ ]:
# TODO: create ROUGE metrics


In [ ]:
# TODO: Evaluate the original model's result


In [ ]:
# TODO: Evaluate with Bleu metrics
